In [ ]:
import pennylane as qml
from pennylane.optimize import AdamOptimizer
from pennylane import numpy as np

import matplotlib.pyplot as plt

In [ ]:
np.random.seed(32)

### Data Generation

In [ ]:
def Line(samples):

    xdata = []
    ydata = []

    for i in range(samples):
        x = np.random.uniform(-1, 1)
        y = x

        xdata.append(x)
        ydata.append(y)
    
    return np.array(xdata, requires_grad = True) , np.array(ydata, requires_grad = False)

# --------------------------------------------------- #

def Exp(samples):
    xdata = []
    ydata = []

    for i in range(samples):
        x = np.random.uniform(-1, 1)
        y = np.exp(x-1)

        xdata.append(x)
        ydata.append(y)
    
    return np.array(xdata, requires_grad = True) , np.array(ydata, requires_grad = False)

# --------------------------------------------------- #

def Inverse(samples):
    xdata = []
    ydata = []
    
    for i in range(samples):
        xarr = np.arange(-1, 1.02, 1/50)
        x = np.random.choice(xarr)
        y = 1 / (50*x)

        xdata.append(x)
        ydata.append(y)
    
    return np.array(xdata, requires_grad = True) , np.array(ydata, requires_grad = False)

# --------------------------------------------------- #

def Square(samples):
    xdata = []
    ydata = []
    
    for i in range(samples):
        x = np.random.uniform(-1, 1)
        y = x**2

        xdata.append(x)
        ydata.append(y)
    
    return np.array(xdata, requires_grad = True) , np.array(ydata, requires_grad = False)

def Cos(samples):
    xdata = []
    ydata = []

    for i in range(samples):
        x = np.random.uniform(-np.pi, np.pi)
        y = np.cos(x)

        xdata.append(x)
        ydata.append(y)
    
    return np.array(xdata, requires_grad = True) , np.array(ydata, requires_grad = False)

### VQC Model

In [ ]:
dev = qml.device("lightning.qubit", wires = 10)

@qml.qnode(dev)
def DataReup_model(x, params, num_qubits, num_layers):
    
    idx = 0

    for _ in range(num_layers):
        for i in range(num_qubits):
            qml.RY(params[idx], wires = i)
            qml.RZ(params[idx+1]*x, wires = i, id= 'Data Upload')
            qml.RZ(params[idx+2], wires = i)
            qml.RY(params[idx+3], wires = i)
            qml.RZ(params[idx+4], wires = i)
            idx += 5

        if num_qubits > 1:
            for q in range(0,num_qubits-1,1):
                qml.CZ([q, q+1])
                
            if num_qubits > 2:
                qml.CZ([num_qubits-1, 0])

    for i in range(num_qubits):
        qml.RY(params[-1-i], wires = i)
    
    obs = qml.PauliZ(0)
    for i in range(num_qubits-1):
        obs @= qml.PauliZ(i+1)

    return qml.expval(obs)

### Cost function

In [ ]:
def cost_function(xdata, thetas, num_qubits, num_layers, ydata):
    ypred = [qml.grad(DataReup_model)(x_, thetas, num_qubits, num_layers)[0] for x_ in xdata]
    loss = 0

    for i in range(len(ydata)):
        loss += (ydata[i] - ypred[i]) ** 2

    return loss / len(ydata)

### Model running

In [ ]:
def model_running(xdata, thetas, num_qubits, num_layers):
    exp = [DataReup_model(x_, thetas, num_qubits, num_layers) for x_ in xdata]
    diffpred = [qml.grad(DataReup_model)(x_, thetas, num_qubits, num_layers)[0] for x_ in xdata]

    return np.array(exp), np.array(diffpred)

In [ ]:
def iterate_minibatches(inputs, targets, batch_size):

    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

### Utility

In [ ]:
def accuracy(ypred, ydata):

    score = 0
    for i in range(len(ydata)):
        acc = ypred[i] / ydata[i] - 1
        if np.abs(acc) < 0.2:
            score += 1

    return score/len(ydata)

-----

In [ ]:
train_data, train_target = Line(50)
test_data, test_target = Line(100)

# Quantum circuit settings
num_qubits = 2
num_layers = 4

# Trainnig option settings
epochs = 100
batch_size = 25
lr = 0.05

# Using the Optimizer
opt = AdamOptimizer(lr)                 ### Adam Optimizer

# Initializing random parameters for the circuit
thetas = np.random.uniform(size = 5*num_qubits*num_layers+num_qubits, requires_grad = True)
#print(thetas)

In [ ]:
### Evaluating the qNN
# Running the model with val data
exp_train, diffpred_train = model_running(train_data, thetas, num_qubits, num_layers)
score_train = accuracy(diffpred_train, train_target)

# Running the model with the test data
exp_test, diffpred_test = model_running(test_data, thetas, num_qubits, num_layers)
score_test = accuracy(diffpred_test, test_target)

# Saving predictions with random weights for comparison 
initial_expectation = exp_test
initial_diffpred = diffpred_test

loss = cost_function(test_data, thetas, num_qubits, num_layers, test_target)

loss_list = [loss]
accuracy_train_list = [score_train]
accuracy_test_list = [score_test]

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
        0, loss.item(), score_train, score_test
    )
)

for it in range(epochs):
    for Xbatch, ybatch in iterate_minibatches(train_data, train_target, batch_size=batch_size):
        _, thetas, _, _, _ = opt.step(cost_function, Xbatch, thetas, num_qubits, num_layers, ybatch)

    exp_train, diffpred_train = model_running(train_data, thetas, num_qubits, num_layers)
    score_train = accuracy(diffpred_train, train_target)
    loss = cost_function(test_data, thetas, num_qubits, num_layers, test_target)

    exp_test, diffpred_test = model_running(test_data, thetas, num_qubits, num_layers)
    score_test = accuracy(diffpred_test, test_target)
    res = [it + 1, loss.item(), score_train, score_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )

    loss_list.append(loss)
    accuracy_train_list.append(score_train)
    accuracy_test_list.append(score_test)

In [ ]:
print("Learned weights")
print("thetas = {}".format(thetas))

fig, axis = plt.subplots(1, 3, figsize=(10, 3))

axis[0].scatter(test_data, test_target, s=2, label = "Target")
axis[1].scatter(test_data, test_target, s=2, label = "Target")
axis[1].scatter(test_data, initial_diffpred, color = 'red', marker = '.', label = "differentiate x")
axis[1].scatter(test_data, initial_expectation, color = 'c', marker = 'x', label = 'expecatation value')
axis[2].scatter(test_data, test_target, s=2, label = "Target")
axis[2].scatter(test_data, diffpred_test, color = 'red', marker = '.', label = "differentiate x")
axis[2].scatter(test_data, exp_test, color = 'c', marker = 'x', label = 'expecatation value')

axis[0].set_ylim((-1,1))
axis[1].set_ylim((-1,1))
axis[2].set_ylim((-1,1))

axis[0].grid(True)
axis[1].grid(True)
axis[2].grid(True)

axis[0].legend()
axis[1].legend()
axis[2].legend()

axis[0].set_title("Target Integrand")
axis[1].set_title("Initial Distribution")
axis[2].set_title("Trained Distribuition")

axis[0].grid(True)
plt.show()

In [ ]:
def Integration(xmin, xmax, thetas, num_qubits,num_layers):
    upper = DataReup_model(xmax, thetas, num_qubits, num_layers)
    lower = DataReup_model(xmin, thetas, num_qubits, num_layers)
    
    return upper - lower

In [ ]:
xlin = np.linspace(-np.pi, np.pi)
VQC_integral = Integration(0, xlin, thetas, num_qubits, num_layers)

fig, axis = plt.subplots(
    2, 1, sharex=True, figsize=(6, 6 * 6 / 8), gridspec_kw={"height_ratios": [5, 2]}
)
axis[0].set_title(f"Approximation with {num_qubits} qubits & {num_layers} layers")
axis[0].plot(xlin, VQC_integral, '--',label = "VQC Prediction")

### Please fill the integration function with the constant we can make it with hand
axis[0].plot(xlin, np.sin(xlin), 'r-.' ,label = "Target")

axis[0].legend()
axis[0].grid(True)
axis[0].set_xlabel('x')
axis[0].set_ylabel('Distribution values')

axis[1].plot(xlin, VQC_integral-np.sin(xlin), label = 'Difference')
axis[1].hlines(0, -np.pi, np.pi, color="black", alpha=0.7, ls="-.", lw=1.5)
axis[1].set_ylim(-0.1,0.1)
axis[1].set_ylabel("Pred-Target")
axis[1].set_xlabel('x')
axis[1].grid(False)